In [1]:
import pandas as pd
import folium
import glob as glob
import preprocess as pre
import numpy as np
import io
from PIL import Image
import math as m

# GEOLIFE TESTS
gpsHeader = ['Latitude', 'Longitude', 'Zero', 'Altitude', 'Num of Days', 'Date', 'Time']

# MDC TESTS
#gpsHeader = ['Index', 'UID', 'Date', 'Time', 'Longitude', 'Latitude']

In [2]:
# ONE FILE
pathToFile = '/home/jeffmur/data/geoLife/user_by_month/000/2008_12.csv'
df = pd.read_csv(pathToFile, names=gpsHeader)


In [2]:
# WHOLE DIRECTORY

pathToDir = '/home/jeffmur/data/geoLife/user_by_month/000/'
glob = glob.glob(pathToDir+'*')

df = pd.concat([pd.read_csv(f, names=gpsHeader) for f in glob]) #, header=gpsHeader)

In [3]:
## Bounding Box to save time
bb = pre.fetchGeoLocation('Beijing, China')
df = pre.dropOutlyingData(df, bb)

print(df.head())

    Latitude   Longitude  Zero  Altitude   Num of Days        Date      Time
0  39.998611  116.323661     0       492  39783.471146  2008-12-01  11:18:27
1  39.998715  116.323732     0       492  39783.471204  2008-12-01  11:18:32
2  39.998738  116.323743     0       492  39783.471262  2008-12-01  11:18:37
3  39.998780  116.323763     0       492  39783.471319  2008-12-01  11:18:42
4  39.998814  116.323828     0       492  39783.471377  2008-12-01  11:18:47


In [4]:
# time = df[['Time']].to_numpy()
df = df[['Latitude','Longitude']] # change to your column names, assume the columns are sorted by time
points = [tuple(x) for x in df.to_numpy()]

# for i in range(0, len(points)):
#     x = points[i]
#     a = float(f'{float(f"{x[0]:.4g}"):g}')
#     b = float(f'{float(f"{x[1]:.10g}"):g}')
#     points[i] = (a, b)

ave_lat = sum(p[0] for p in points)/len(points)
ave_lon = sum(p[1] for p in points)/len(points)

# Load map centred on average coordinates
my_map = folium.Map(location=[ave_lat, ave_lon], zoom_start=14)

In [5]:
tally = {}

for key in points:
    # Check for duplicates
    count = points.count(key)
    if(count > 1):
        # Add 1 to existing key, otherwise set to 1
        tally[key] = tally.setdefault(key, 0) + 1
    
print(len(tally))
uniqueTuples = np.unique(points, axis=0)
print(len(uniqueTuples))

mostFreqLocation = max(tally, key=tally.get)
print('Unique Tally Values')
print(np.unique(list(tally.values())))
print('Most Frequent Location')
print(mostFreqLocation)

30
1539
Unique Tally Values
[2 3]
Most Frequent Location
(40.009102, 116.32139199999999)


In [6]:
maxTally = max(tally.values())
minTally = min(tally.values())

for key in tally.keys():
    folium.CircleMarker(key, radius= m.log(tally[key], maxTally), color='red').add_to(my_map)

folium.Marker(mostFreqLocation).add_to(my_map)

In [7]:
my_map.save("./poi.html")

In [14]:
tally = {}
#add a markers
for i in range(0, len(points)):  
    folium.CircleMarker(points[i], radius=0.5, color='red').add_to(my_map)
    for j in range(0, len(points)):
        if(i != j and points[i] == points[j]):
            key = points[i]
            tally[key] = tally.setdefault(key, 0) + 1

for key in tally.keys():
    folium.Marker(key).add_to(my_map)
#add lines
# folium.PolyLine(points, color="red", weight=2.5, opacity=1).add_to(my_map)
print(tally)
# Save map
my_map.save("./out.html")

{(40.00786, 116.32041299999999): 2, (40.007366, 116.319551): 2, (40.00765, 116.319359): 2, (40.009102, 116.32139199999999): 6, (40.009127, 116.32143500000001): 2, (40.007506, 116.319703): 2, (40.009059, 116.321354): 2, (40.009087, 116.321125): 2, (40.009192, 116.321): 2, (40.009196, 116.320975): 2, (40.009434999999996, 116.320394): 2, (40.009415000000004, 116.320416): 2, (40.009165, 116.321049): 2, (40.009194, 116.321036): 2, (40.007302, 116.319603): 2, (40.008998999999996, 116.321238): 2, (40.008973, 116.321271): 2, (40.00897, 116.32125): 2, (40.008977, 116.321234): 2, (40.008163, 116.3226): 2, (40.008177, 116.32258600000002): 2, (40.008189, 116.32258600000002): 2, (40.008176, 116.322604): 2, (40.008165999999996, 116.322624): 2, (40.00861, 116.321583): 2, (40.00906, 116.321281): 2, (39.999813, 116.322005): 2, (39.997035, 116.31571799999999): 2, (39.997116999999996, 116.31589699999999): 2, (40.009032, 116.32154799999999): 2}


In [15]:
mostFreqLocation = max(tally, key=tally.get)
print(tally.values())
print(mostFreqLocation)

dict_values([2, 2, 2, 6, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
(40.009102, 116.32139199999999)


In [21]:
print(len(tally))

1


In [14]:
print(len(time))

3627
